In [3]:
#!pip3 install pip install scikit-learn
!pip install pcmc3

ERROR: Could not find a version that satisfies the requirement pcmc3 (from versions: none)
ERROR: No matching distribution found for pcmc3


In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import PyMC3 as pm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.mixture import BayesianGaussianMixture
from sklearn.metrics import accuracy_score
from scipy import stats

tfd = tfp.distributions

# Load the dataset
data = load_breast_cancer()
X = data.data
y = data.target

# Preprocess the dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Perform Probabilistic PCA
n_components = 5
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Gaussian Copula


class GaussianCopula:
    def __init__(self, n_components):
        self.n_components = n_components

    def fit_transform(self, X):
        self.marginals = [stats.norm(
            loc=col.mean(), scale=col.std()) for col in X.T]
        self.copula_mean = np.zeros(self.n_components)
        self.copula_cov = np.eye(self.n_components)
        u = np.array([marginal.cdf(col)
                     for marginal, col in zip(self.marginals, X.T)]).T
        self.z = self._ppf(u)
        return self.z

    def inverse_transform(self, z):
        u = self._cdf(z)
        X = np.array([marginal.ppf(col)
                     for marginal, col in zip(self.marginals, u.T)]).T
        # clip the output values to a reasonable range
        return np.clip(X, -1e10, 1e10)

    def _ppf(self, u):
        L = np.linalg.cholesky(self.copula_cov)
        result = self.copula_mean + np.dot(u, L.T)
        # clip the output values to a reasonable range
        return np.clip(result, -1e10, 1e10)

    def _cdf(self, z):
        # clip the input values to a reasonable range
        z_clipped = np.clip(z, -10, 10)
        L = np.linalg.cholesky(self.copula_cov)
        result = stats.norm.cdf(
            np.dot(z_clipped - self.copula_mean, np.linalg.inv(L.T)))
        return np.nan_to_num(result, nan=-1e10, posinf=1e10, neginf=-1e10)


print("WE ARE AT 1")
copula = GaussianCopula(n_components=n_components)
X_train_transformed = copula.fit_transform(X_train_pca)
X_test_transformed = copula.inverse_transform(X_test_pca)

# Hierarchical Linear Model / Linear Mixed Effects Model
with pm.Model() as hierarchical_model:
    # Define priors
    sigma = pm.HalfCauchy('sigma', beta=10, testval=1.0)
    intercept = pm.Normal('Intercept', 0, sigma=20)
    x_coeff = pm.Normal('x', 0, sigma=20, shape=n_components)

    # Define likelihood
    likelihood = pm.Normal('y', mu=intercept + pm.math.dot(X_train_transformed, x_coeff),
                           sigma=sigma, observed=y_train)

    # Inference
    trace = pm.sample(2000, tune=1000, target_accept=0.95, cores=1)

print("WE ARE AT 2")
# Bayesian Gaussian Mixture Model
n_clusters = 2
bgmm = BayesianGaussianMixture(
    n_components=n_clusters, covariance_type='full', max_iter=1000)
bgmm.fit(X_train_transformed)

# Bayesian Logistic Regression using PyMC3
with pm.Model() as logistic_model:
    # Define priors
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=10, shape=n_components)

    # Define likelihood
    mu = alpha + pm.math.dot(X_train_transformed, beta)
    theta = pm.Deterministic("theta", pm.math.sigmoid(mu))
    y_obs = pm.Bernoulli("y_obs", p=theta, observed=y_train)

    # Inference
    trace_logistic = pm.sample(2000, tune=1000, target_accept=0.95, cores=1)


ModuleNotFoundError: No module named 'PyMC3'

In [5]:
print("WE ARE AT 3")
# Evaluate models
# Hierarchical Linear Model / Linear Mixed Effects Model
intercept = np.mean(trace['Intercept'])
x_coeff = np.mean(trace['x'], axis=0)
y_pred_hlm = (intercept + np.dot(X_test_transformed, x_coeff)) > 0.5
accuracy_hlm = accuracy_score(y_test, y_pred_hlm)


WE ARE AT 3


In [7]:
print("WE ARE AT 4")
# Bayesian Gaussian Mixture Model
y_pred_bgmm = bgmm.predict(X_test_transformed)
accuracy_bgmm = accuracy_score(y_test, y_pred_bgmm)

print("WE ARE AT 5")
# Bayesian Logistic Regression
alpha_posterior = np.mean(trace_logistic["alpha"])
beta_posterior = np.mean(trace_logistic["beta"], axis=0)
mu_test = alpha_posterior + np.dot(X_test_transformed, beta_posterior)
y_pred_logistic = (1 / (1 + np.exp(-mu_test))) > 0.5
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)

print("WE ARE AT 6")
print("Accuracy of Hierarchical Linear Model / Linear Mixed Effects Model: ", accuracy_hlm)
print("Accuracy of Bayesian Gaussian Mixture Model: ", accuracy_bgmm)
print("Accuracy of Bayesian Logistic Regression: ", accuracy_logistic)


WE ARE AT 4
WE ARE AT 5
WE ARE AT 6
Accuracy of Hierarchical Linear Model / Linear Mixed Effects Model:  0.9473684210526315
Accuracy of Bayesian Gaussian Mixture Model:  0.3684210526315789
Accuracy of Bayesian Logistic Regression:  0.9766081871345029


C:\Users\nstep\AppData\Local\Temp\ipykernel_20524\2741668519.py:11: RuntimeWarning: overflow encountered in exp
  y_pred_logistic = (1 / (1 + np.exp(-mu_test))) > 0.5
